In [1]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.multioutput import MultiOutputRegressor

# Load Datasets

In [2]:
def load_data(file_path):
    A = np.loadtxt(file_path)
    X = A[:, :9]    # Input features
    y = A[:, 9:]    # Output labels
    return X, y

In [3]:
# Load all 3 datasets
X_final, y_final = load_data('data/tictac_final.txt')
y_final = y_final[:, 0]

X_single, y_single = load_data('data/tictac_single.txt')
y_single = y_single[:, 0]

X_multi, y_multi = load_data('data/tictac_multi.txt')

## Multilayer Perceptron Classifier

In [4]:
def mlp_clf_train(X, y):
    # Split into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

    from sklearn.preprocessing import StandardScaler 
    scaler = StandardScaler()
    scaler.fit(X_train) 
    X_train = scaler.transform(X_train)  
    X_test = scaler.transform(X_test)
    
    # Define and train model
    model = MLPClassifier(max_iter=300)
    model.fit(X_train, y_train)

    # Get cross validation accuracy
    val_accuracy = cross_val_score(model, X_train, y_train, cv=10, scoring="accuracy")
    val_accuracy = np.mean(val_accuracy)

    # Get test accuracy
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    # Get confusion matrix
    confusion_mtrx = confusion_matrix(y_test, y_pred, normalize="true")

    return val_accuracy, test_accuracy, confusion_mtrx

In [5]:
def print_clf_results(val_accuracy, test_accuracy, confusion_mtrx, dataset_name):
    print(f"Performance of Multilayer Perceptron Classification on {dataset_name}:")
    print("Cross Validation Accuracy = ", val_accuracy)
    print("Test Accuracy = ", test_accuracy)
    print("Confusion Matrix:")
    print(confusion_mtrx.round(decimals=3)) # Round to 3 decimal places

## Final Dataset

In [6]:
val_acc_final, test_acc_final, confusion_mtrx_final = mlp_clf_train(X_final, y_final)
print_clf_results(val_acc_final, test_acc_final, confusion_mtrx_final, "Final Dataset")

/Users/justingalin/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Performance of Multilayer Perceptron Classification on Final Dataset:
Cross Validation Accuracy =  0.9869617224880383
Test Accuracy =  0.9739583333333334
Confusion Matrix:
[[0.925 0.075]
 [0.    1.   ]]


## Single Dataset

In [7]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
val_acc_single, test_acc_single, confusion_mtrx_single = mlp_clf_train(X_single, y_single)
print_clf_results(val_acc_single, test_acc_single, confusion_mtrx_single, "Single Dataset")

/Users/justingalin/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/justingalin/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/justingalin/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/justingalin/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/justi

Performance of Multilayer Perceptron Classification on Single Dataset:
Cross Validation Accuracy =  0.875381679389313
Test Accuracy =  0.8832951945080092
Confusion Matrix:
[[0.916 0.003 0.012 0.012 0.028 0.    0.012 0.006 0.009]
 [0.03  0.845 0.024 0.012 0.03  0.03  0.012 0.006 0.012]
 [0.016 0.016 0.872 0.032 0.027 0.    0.016 0.011 0.011]
 [0.026 0.    0.009 0.88  0.026 0.    0.017 0.009 0.034]
 [0.045 0.005 0.015 0.    0.916 0.    0.015 0.    0.005]
 [0.    0.066 0.013 0.    0.    0.882 0.026 0.013 0.   ]
 [0.03  0.051 0.051 0.071 0.    0.01  0.788 0.    0.   ]
 [0.    0.04  0.    0.06  0.02  0.    0.    0.88  0.   ]
 [0.034 0.022 0.045 0.    0.    0.    0.    0.    0.899]]


/Users/justingalin/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


# Multilayer Perceptron Regressor

In [8]:
def MLP_reg_train(X, y):
    # Split into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
    from sklearn.preprocessing import StandardScaler 
    scaler = StandardScaler()
    scaler.fit(X_train) 
    X_train = scaler.transform(X_train)  
    X_test = scaler.transform(X_test)
    
    # Define and train model
    base_regressor = MLPRegressor()
    model = MultiOutputRegressor(base_regressor)
    model.fit(X_train, y_train)

    # Get cross validation accuracy
    val_rmse = cross_val_score(model, X_train, y_train, cv=10, scoring="neg_mean_squared_error")
    val_rmse = np.mean(val_rmse * -1)  # Convert to positive

    # Get test accuracy
    test_accuracy = model.score(X_test, y_test)

    # Get RMSE
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    return val_rmse, test_accuracy, rmse

In [9]:
def print_reg_results(val_rmse, test_accuracy, rmse, dataset_name):
    print(f"Performance of MLP Regression on {dataset_name}:")
    print("Cross Validation RMSE = ", val_rmse)
    print("Test Accuracy = ", test_accuracy)
    print("RMSE = ", rmse)

## Multi Dataset

In [10]:
val_rmse_multi, test_acc_multi, rmse_multi = MLP_reg_train(X_multi, y_multi)
print_reg_results(val_rmse_multi, test_acc_multi, rmse_multi, "Multi Dataset")

/Users/justingalin/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/justingalin/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/justingalin/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/justingalin/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/justi

Performance of MLP Regression on Multi Dataset:
Cross Validation RMSE =  0.052368481463396496
Test Accuracy =  0.6827192724419846
RMSE =  0.2288027429324887
